In [1]:
import requests
import pandas as pd
import numpy as np
import regex as re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
# K-means
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
#Import the previously save spotify with cluster where the spotify songs are already divided in clusters.
spotify_with_cluster = pd.read_csv('spotify_with_cluster.csv')
#print(spotify_with_cluster.head())

### Song Recommender
- Ask for user input
- If song present in the list, then recommend another song from the list
- If not, a polite message

In [3]:
def song_recommender(title, artist):
    #Convert the input to lower case
    title_input = remove_special_characters(title.lower())
    artist_input = remove_special_characters(artist.lower())
    
    if (artist_input):
        # Check if the input combination is present in the dataframe
        result = top_songs_lower[(top_songs_lower['Song'].values == title_input) & (top_songs_lower['Artist'].values == artist_input)]
        #print(result)
    else:
        result = top_songs_lower[(top_songs_lower['Song'].values == title_input)]

    if not result.empty:
        #Get the indices
        indices_to_remove = result.index
        temp_top_songs = top_songs.drop(indices_to_remove)

        rec_song = temp_top_songs.sample().squeeze()
        print(f"You might also like: {rec_song['Song']} by {rec_song['Artist']}")
    else:
        #Opening the spotify connection
        print("Connecting to Spotify..")
        
        #If artist input is empty, the query will be modified accordingly.
        if (artist_input):        
            #querying spotify
            results = sp.search(q="artist:" + artist_input + " track:" + title_input, type="track")
        else:
            results = sp.search(q="track:" + title_input, type="track")
        
        #print(results['tracks']['total'])

        #First check if the query returns results (i.e total>0)
        if results['tracks']['total']!=0 :
            
            #Iterates through the results list and checks with the user if the song is what they intended.
            index = 0
            track_id=''
            while index < results['tracks']['total']:
                inp = input(f"Did you mean: {results['tracks']['items'][index]['name']} by {results['tracks']['items'][index]['artists'][0]['name']}")
                #track_id = results['tracks']['items'][index]['id']
                if (inp in ('yes', 'y','ye')):
                    track_id = results['tracks']['items'][index]['id'] #Id of the first track
                    break
                else: #if not, then loop again.
                    index+=1
                    print("Let's try again!")
            
            #Getting the audio features using the id
            new_data = sp.audio_features(track_id)
            data_df = json_normalize(new_data[0]) #normalizing the data to a df
            num_data_df = data_df.select_dtypes(include='number') #getting the numerical data only to fit
            #print(num_data_df)
            
            #Transform the data using the Standard Scaler that used to fit and the same kmeans to predict the cluster
            X_prep = scaler.transform(num_data_df) 
            predicted_cluster = kmeans.predict(X_prep)
            cluster = predicted_cluster[0]
            
            #Getting songs only from that cluster and squeeze just the top one
            rec_song = spotify_with_cluster[spotify_with_cluster['cluster']== cluster].sample().squeeze()

            print(f"You might also like: {rec_song['song_and_artist']}\n")
        else:
            print("The song is not in Spotify.")

def remove_special_characters(text):
    # Use a regular expression to replace non-alphanumeric characters with an empty string
    return ''.join(e for e in text if e.isalnum() or e.isspace())

def open_spotify_connection():
    #Opens a new spotify connection
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()

    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [4]:
# Getting all lower str
top_songs = pd.read_csv('top_songs.csv')
top_songs_lower = top_songs.apply(lambda x: x.astype(str).str.lower())
top_songs_lower['Song'] = top_songs_lower['Song'].apply(remove_special_characters)
#top_songs_lower

### Opening spotify connection, loading the scaler and kmean using pickle.

In [5]:
sp = open_spotify_connection()
#Using pickle to get the existing scaler and kmean
kmeans = pickle.load(open('kmean.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl','rb'))

### Getting User Input

In [ ]:
#Get user input
loop = True

print("Welcome to Song Recommender! \n")
while loop:   
    title_input = input('Please Enter the Song Title: ')
    
    #At least the title has to be entered
    if (title_input):
        artist_input = input('Please Enter the Artist Name: ')
        print('\n')
        song_recommender(title_input, artist_input)
    else:
        print("Song title cannot be empty")
        
    qn = input('Would you like to try again (Yes or No)? \n')

    #If the user enters anything other than Yes, the loop will break
    if (qn.lower() not in ('yes', 'y', 'ye')):
        print("Thank you. Have a nice day!")
        loop = False

        

Welcome to Song Recommender! 

Please Enter the Song Title: hello
Please Enter the Artist Name: 


Connecting to Spotify..
Did you mean: Hello by Adelen
Let's try again!
Did you mean: Hello (feat. A Boogie Wit da Hoodie) by Pop Smoken
Let's try again!
Did you mean: Hello? by Clairoy
You might also like: Leaning by The KVB

Would you like to try again (Yes or No)? 
y
Please Enter the Song Title: unstoppable
Please Enter the Artist Name: 


You might also like: I Don't Care by Ed Sheeran and Justin Bieber
